In [1]:
import sys, platform
sys.path.append('/home/jsong/notebooks')

import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import datetime as dt
import dask
from dateutil import parser
import velidb.btcommon as bt
from velidb.tobcache import tob_cache

pd.set_option('display.mpl_style', 'default')
%matplotlib inline

# user defined classes -- accessorys function
import DATA          as my_data
import VISUALIZER    as my_visualizer
# user defined classes -- different modemls for BTMU project
import MODEL_RANDOM  as my_model_RANDOM
import MODEL_MA      as my_model_MA
import MODEL_CROSS   as my_model_CROSS
import CONFIGURATION as my_config

feeds_list = ['BTMU_HC_1', 'EBS_ULTRA_NY_A', 'EBS_LIVE_NY_XML_A']
currency_list = ['EURUSD']
# currency = ['EURUSD', 'GBPUSD', 'USDJPY', 'USDHCHF', 'EURCHF', 'AUDUSD', 'USDCAD', 'NZDUSD', 'USDMZN', 'EURSEK',
#             'USDHKD', 'USDZAR', 'EURNOK', 'EURCZK', 'EURDKK', 'EURHUF', 'USDCNH', 'EURPLN']

# configuration (time scope, feeds, currency etc.)
my_debug_plot = False       # 
my_debug_data = True
my_start_time = dt.datetime(2017, 4, 4)  # start date in (Year, Month, Date)
my_days       = 1            # number of days that data span over
my_debug_hours = 0            # hours of data (can be used for fast debug)
my_debug_minutes = 0          # minutes of data (can be used for fast debug)
my_debug_seconds = 5          # seconds of data (can be used for fast debug)
my_sampling_rule = '100L'    # for resample, H, T, S, L, U, N (from hour to ns). Example: 100L is 100ms
my_sliding_window = 1        # for rolling MA (1 unit)
my_computing_window = 30  # for periodic chunk size (30000 is 30000ms, which is 30s)

# the main function that does the computation
def run():
    ######################
    # Model Configuration
    ######################
    # sampling_rule     : the time window size for resample, ''  means the raw data                     
    # sliding_window    : the window size for moving average computation
    # computing_period  : the periodicity of computing the MA crossing, in ms    
    # Example:      300/100L => 3 points
    #               if sampling_rule is 100ms, data_period is 300ms
    #               then there will be 3 points/signals generated per period
    # Note: data_period should be automatically measured (i.e., 95%)
    # For the rest parameters, see above comments
    config = my_config.ModelConfiguration(sampling_rule=my_sampling_rule,   
                                          sliding_window=my_sliding_window,       
                                          computing_window=my_computing_window, 
                                          start_time=my_start_time,
                                          days=my_days,
                                          hours=my_debug_hours,
                                          minutes=my_debug_minutes,
                                          seconds=my_debug_seconds,                                          
                                          feeds=feeds_list,
                                          currency=currency_list,
                                          debug_plot=my_debug_plot,
                                          debug_data=my_debug_data)

    # load all the data, and do the conversion if necessary
    # from the preview, it seems that 15 ms is the period
    data = my_data.Init_Data(config)

    # plot the data
    if config.debug_plot:
        viz = my_visualizer.Visualizer()    
        viz.preview(data)

    # input 2 different feed streams and generate signals from 2 input feed streams
    # prepare the data stream
    btmu_stream       = my_data.DataStream(data.dfs['BTMU_HC_1'], 'BTMU_HC_1', config)
    ebs_ultra_stream  = my_data.DataStream(data.dfs['EBS_ULTRA_NY_A'], 'EBS_ULTRA_NY_A', config)
    ebs_live_stream   = my_data.DataStream(data.dfs['EBS_LIVE_NY_XML_A'], 'EBS_LIVE_NY_XML_A', config)
    
    # BTMU vs EBS_ULTRA
    model = my_model_CROSS.MovingAverageCrossStrategy(btmu_stream, ebs_ultra_stream, config)
    signals = model.generate_signals()    
    
    c1 = signals['bid_signal'].map(lambda x: x == 1)
    
    print(c1.shape)    
        
    #c2 = btmu_stream.data['bid'].loc[btmu_ebs_ultra_signals[c1].index]
    # plot and print some information here    
    #print(signals.head(5))
    #print(np.count_nonzero(signals.signal))
    plt.plot(signals.BTMU_HC_1_mavg_bid, c='red')
    plt.plot(signals.EBS_ULTRA_NY_A_mavg_bid, c='blue')
    #plt.scatter(signals[c1].index, signals[c1].BTMU_HC_1_mavg_bid, c='black', s=10, edgecolor='none')

    plt.title('MA Cross')
    plt.xlabel('Time')
    plt.ylabel('MA Price')
    plt.legend(loc='lower left')
    plt.show() 
    

if __name__ == "__main__":
    print ('Starting...')
    print ("python version: " + platform.python_version())
    print ("pandas version: " + pd.__version__)
    print ("numpy  version: " + np.__version__)
    print ("dask  version: " + dask.__version__)

    run()
    
    print ('Done!!')

/opt/rh/python33/root/usr/lib/python3.3/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


Starting...
python version: 3.3.2
pandas version: 0.18.0
numpy  version: 1.11.1
dask  version: 0.14.2


start computing... bid
This is mask
<class 'numpy.ndarray'>
[ True False False False False False False False False]
(9,)
This is masked data
<class 'pandas.core.series.Series'>
timestamp
2017-04-04 00:00:00.742159104    1.06644
2017-04-04 00:00:02.697229056    1.06645
2017-04-04 00:00:02.697229056    1.06645
2017-04-04 00:00:02.697229056    1.06645
2017-04-04 00:00:02.697229056    1.06645
2017-04-04 00:00:04.918800896    1.06647
2017-04-04 00:00:04.918800896    1.06647
2017-04-04 00:00:04.918800896    1.06647
2017-04-04 00:00:04.918800896    1.06647
Name: bid, dtype: float64
timestamp
2017-04-04 00:00:00.742159104    1.06644
Name: bid, dtype: float64
(1,)
This is mask
<class 'list'>
[False, True, True, True, True, True, True, True, True]


AttributeError: 'list' object has no attribute 'shape'

In [2]:
%time print("test")

test
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 58.4 µs
